# 因果语言模型训练实例

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

## Step1 导入相关包

In [2]:
from datasets import load_dataset, Dataset
# AutoModelForMaskedLM改成了AutoModelForCausalLM，因果语言模型可以用DataCollatorForLanguageModeling也可以用DataCollatorForSeq2Seq
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, BloomForCausalLM

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("./wiki_cn_filtered/")

In [4]:
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [5]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## Step3 数据集处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("/data/PLM/bloom-1b4-zh")

def process_func(examples):
    contents = [e + tokenizer.eos_token for e in examples["completion"]]
    return tokenizer(contents, max_length=384, truncation=True)

In [7]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10000
})

In [8]:
from torch.utils.data import DataLoader

# 因果语言模型的预训练会自动生成labels，虽然也只是把input_ids重复了一遍！而且都要计算loss
# 微调的时候可不是这样，原始labels以外的部分都要改成-100（[EOS]除外），包括input_ids和padding
dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=False))

In [9]:
next(enumerate(dl)) # 这个batch中第一条被left padding，eos token存在；第二条被right truncation，eos token不存在。这个似乎并没有什么关系

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(0,
 {'input_ids': tensor([[    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3

In [10]:
tokenizer.pad_token, tokenizer.pad_token_id

('<pad>', 3)

In [11]:
tokenizer.eos_token, tokenizer.eos_token_id

('</s>', 2)

## Step4 创建模型

In [12]:
model = AutoModelForCausalLM.from_pretrained("/data/PLM/bloom-1b4-zh")
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(46145, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (l

## Step5 配置训练参数

In [13]:
args = TrainingArguments(
    output_dir="./causal_lm",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    fp16=True
)

## Step6 创建训练器

In [14]:
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds,
    # 上面那个DataCollatorForLanguageModeling只是取样查看，真正的DataCollatorForLanguageModeling要用在这！
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Step7 模型训练

In [15]:
trainer.train()

Step,Training Loss
10,3.445200
20,3.338100
30,3.360900
40,3.310500
50,3.283800
60,3.310800
70,3.271400
80,3.275400
90,3.246600
100,3.264300


TrainOutput(global_step=312, training_loss=3.214142547203944, metrics={'train_runtime': 853.1351, 'train_samples_per_second': 11.721, 'train_steps_per_second': 0.366, 'total_flos': 2.685584078733312e+16, 'train_loss': 3.214142547203944, 'epoch': 1.0})

## Step8 模型推理

In [16]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [17]:
pipe("西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安", max_length=128, do_sample=True)

[{'generated_text': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安市经开区的综合性博物馆，位于西安交通大学校区内的科技大楼内。博物馆于2020年9月8日随交通大学百年校庆正式开放，馆内共分为9个常设展厅、6个临时展厅。该博物馆隶属于西安交通大学博物馆研究部。\n院史馆\n展厅号: W1\n简介:\n院史馆位于西安交通大学科技大楼二楼，共2层，总展览面积约300平方米。该馆馆名由西安交通大学历史悠久的百年校训“治学严谨、为人敦厚”"}]

In [18]:
pipe("下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常", max_length=128, do_sample=True)

[{'generated_text': '下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常繁荣，游戏产业的产值越来越大，也有游戏厂商为吸引用户，不断推出各种低门槛的游戏，其中《街头足球》和《足球经理》表现优异，但是《FIFA 18》和《FIFA 19》尚未公布，这两款游戏是针对《FIFA 18》和《FIFA 19》游戏发售后的续作而设立的。其中《街头足球》在发售首周就吸引了超过700万用户的关注，虽然《FIFA 18》和《FIFA 19》尚未发行，但是凭借着这两个游戏，已经让游戏市场火了大半年。\n下面是一则游戏游戏新闻。\n'}]